# Exercise 3 - Implementing CMAC – a Cipher-based MAC 

### - Assignment 5 - Stancu Bianca & Tharani Neha -  ###

## Imports

In [2]:
from sage.crypto.mq.rijndael_gf import RijndaelGF
import string
from datetime import datetime

##  Helper functions

In case the received message does not fit, we will need to pad with 1, then 0s.

In [3]:
def messagepad(message, block):
    if(len(message)%block) != 0:
        l = (len(message)//block+1)*block
        message=message+"1"
        while len(message)!=l:
            message=message+"0"
    return message

Function to generate a string of 0s of length b

In [4]:
def generate_zeros(b):
    return "0" * b

Function to convert array to binary string.

In [5]:
def arrayToString(v):
    v = str(v)
    v = v.replace("[","")
    v = v.replace("]","")
    v = v.replace(" ","")
    v = v.replace(",","")
    return v

Function to XOR two blocks, as presented in a previous lab.

In [6]:
def XorBlock(block1, block2):
    l = len(block1);
    if (l != len(block2)):
        raise ValueError, "XorBlock arguments must be same length"
    return [(int(block1[j])+int(block2[j])) % 2 for j in xrange(l)];

Function to generate L for the process of subkey generation.

In [7]:
def L_generation(key,b):
    rgf = RijndaelGF(4, 4)
    zeros = generate_zeros(b)
    L = rgf(zeros, key, format='binary')
    return L

Function to generate the two subkeys.

In [22]:
def generate_subkeys(b,key):
    q = 2^b
    F.<x> = GF(q)
    pol_x = F(x)
    L = L_generation(key,b)
    L = list(L)
    L = L[::-1]
    L = F(L)
    k1 = L*pol_x
    k2 = k1*pol_x
    k1b=k1.polynomial().coefficients(sparse=False)
    k2b=k2.polynomial().coefficients(sparse=False)
    k1 = arrayToString(k1b)
    k2 = arrayToString(k2b)
    k1 = k1.zfill(b)
    k2 = k2.zfill(b)
    return (k1,k2)

CMAC implementation.

In [9]:
def CMAC(key,b,message,Tlen):
    if(Tlen > b):
        raise ValueError, "Tlen must be smaller than block size."
    k1 = generate_subkeys(b,key)[0]
    k2 = generate_subkeys(b,key)[1]
    rgf = RijndaelGF(4, 4)
    ilength = len(message)
    message = messagepad(message,b)
    padded = False
    if len(message)!=ilength:
        padded = True
    prev=""
    c=""
    for i in xrange(0,len(message),b):
        block = message[i:i+b]
        if i == 0:
            c = rgf(block, key, format='binary')
            prev = c
        else:
            xorVal = XorBlock(prev,block)
            if i+b == len(message):
                if padded == True:
                    xorVal = XorBlock(xorVal,k1)
                else:
                    xorVal = XorBlock(xorVal,k2)
            xorVal =arrayToString(xorVal)
            c = rgf(xorVal, key, format='binary')
            prev = c
    return c[0:Tlen]

## Running CMAC

In [24]:
b = 128
k = 128
key = "11111000000101011111010111011100100010101010111100111000111001001010001110000011101000000010110001001110100111000101010010101011"
print "Length of key: " + str(len(key))
message1 = "11010100010001001001001101111111010000011000001001100011110110011011100110101101100110000001011000111111101001110110100111100111110111000000011010110111101101010001001000011011100010000000000111010011101110001000101101011100100001000010011110100011001101000101111000111001" * 3
print "Length of message1:" + str(len(message1))
message2 = "1101010001000100100100110111111101000001100000100110001111011001101110011010110110011000000101100011111110100111011010011110011111011100000001101011011110110101000100100001101110001000000000011101001110111000100010110101110010000100001001111010001100110100"
print "Length of message2:" + str(len(message2))

Length of key: 128
Length of message1:816
Length of message2:256


In [25]:
print "CMAC for message with padding:"
start_time = datetime.now()
cmac = CMAC(key,b,message1,64)
print cmac
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

print "CMAc for message without padding:"
start_time = datetime.now()
cmac = CMAC(key,b,message2,64)
print cmac
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

CMAC for message with padding:
x
x
1111011110110111111101000000011010011001000001101011000001010101
Duration: 0:00:08.137358
CMAc for message without padding:
x
x
1101001100100010101011111110100110111011111001010011011110001111
Duration: 0:00:03.559168
